In [8]:
#Load necessary libraries
from bioservices.kegg import KEGG
import pandas as pd
import progressbar
k = KEGG()

In [9]:
#Load the conversion table between affymetrix and common gene names
conversion_table = pd.read_csv('datasets/conversion_table.txt', delimiter='\t')


In [15]:
#Here, we create a concatination of all the pathways for all the genes in the dataset
file = open("pathways.txt", "w+")
pathway = {}
bar = progressbar.ProgressBar(maxval=conversion_table['gene_name'].size, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
i=0
j=0
for gene in conversion_table['gene_name']:
    temp_pathway = k.get_pathway_by_gene(gene, "hsa") # "hsa" assumes that all the genes corresponds to homo sapiens 
    if temp_pathway != None:
        for key in temp_pathway.keys():
            if key not in pathway.keys():
                file.write(str(j) + "\t" + str(key) +"\t"+ str(temp_pathway[key]+"\n"))
                j = j+1
    i = i+1
    bar.update(i)
bar.finish()
file.close() 

[========================================================================] 100%


In [21]:
#Load the conversion table between affymetrix and common gene names
pathways = pd.read_csv('pathways.txt', delimiter='\t')
pathways.size

145407

In [ ]:
#For all the distinct pathways find all the genes that build a given pathway. 
#Create a file with a list of all the interrelations between the genes in a given pathway.
pfsnet_file = open("pfsnet_pathways.txt","w+")
pfsnet_file.write("pathway\tgene1\tgene2\n")
bar = progressbar.ProgressBar(maxval=pathways.size, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
i=0
for ind in pathways['pathway_id']:
    network = k.parse_kgml_pathway(ind)
    entries = pd.DataFrame.from_dict(network['entries'])
    for item in network['relations']:
        gene1 = entries[entries['id']==item['entry1']]
        gene2 = entries[entries['id']==item['entry2']]
        if gene1['gene_names'].values != None and gene2['gene_names'].values != None:
            gene1 = gene1['gene_names'].values[0].split(','[0])[0]
            gene2 = gene2['gene_names'].values[0].split(','[0])[0]
            pfsnet_file.write(str(ind) +"\t"+ str(gene1)+"\t"+ str(gene2) + "\n")
    i = i+1
    bar.update(i)
bar.finish()
pfsnet_file.close()